In [4]:
def filterBad(dataset, row_limit = -1):
    initial_len = len(dataset)
#     dataset = dataset[(dataset['rising_idx'] != 0)]
#     dataset = dataset[(dataset['falling_idx'] != 0)]
    dataset = dataset[(dataset['falling_idx'] - dataset['rising_idx'] > 100)]
    dataset = dataset[(dataset['falling_idx'] - dataset['rising_idx'] < 500)]
#     dataset = dataset[(dataset['volume'] > 500)]
    dataset = dataset[(dataset['first_val'] < 30) | (dataset['last_val'] < 30)]
#     dataset = dataset[(dataset['first_val'] < 30)]
#     dataset = dataset[(dataset['last_val'] < 30)]
    if (row_limit > 0):
        dataset = dataset[(dataset['row'] < row_limit)]
    final_len = len(dataset)
    print('Rejected %d points (%f%%)' % ((initial_len-final_len), 100.0*(initial_len-final_len)/initial_len))
    return dataset


In [10]:

def dropColumns(dataset,cols):
    import pandas as pd
    dataset = dataset.drop(columns=cols)
    return dataset, [x for x in dataset.columns.values.tolist() if x not in cols]

In [ ]:

def buildDataset(filename):
    import pandas as pd
    column_names = ['fileId', 'row','rising_idx','falling_idx','volume','rising_weight',
                'falling_weight', 'first_val', 'last_val', 'delay']
    
    # raw_dataset =  pd.read_feather('../preprocessing/processed.feather')
#     raw_dataset =  pd.read_feather('/local/ssd/home/mattfel/slac/timetool-ML-models/preprocessing/processed.feather')
    raw_dataset =  pd.read_feather('/scratch/mattfel/data-fs/' + filename)

    dataset = raw_dataset.copy()
    dataset.tail()
    
#     unmodeledColumns = ['fileId']
    unmodeledColumns = []
    dataset = dataset.dropna()
    dataset = dataset.drop(columns=unmodeledColumns)
    return dataset, [item for item in column_names if item not in unmodeledColumns]

In [ ]:

def buildDatasetForLattice():
    import pandas as pd
    column_names = ['fileId', 'row','rising_idx','falling_idx','volume','rising_weight',
                'falling_weight', 'first_val', 'last_val', 'delay']
    
    # raw_dataset =  pd.read_feather('../preprocessing/processed.feather')
#     raw_dataset =  pd.read_feather('/local/ssd/home/mattfel/slac/timetool-ML-models/preprocessing/processed.feather')
    raw_dataset =  pd.read_feather('/scratch/mattfel/data-fs/processed_floor.feather')

    dataset = raw_dataset.copy()
    dataset.tail()
    
    unmodeledColumns = ['fileId', 'volume', 'rising_weight', 'falling_weight']
    dataset = dataset.dropna()
    dataset = dataset.drop(columns=unmodeledColumns)
    return dataset, [item for item in column_names if item not in unmodeledColumns]

In [7]:
def splitDataset(dataset, split):
    from sklearn.model_selection import train_test_split
    X, y = dataset.iloc[:,:-1],dataset.iloc[:,-1]
    train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=123)
    return train_dataset, test_dataset, train_labels, test_labels

In [1]:
def splitDatasetLabels(dataset, labels, split):
    from sklearn.model_selection import train_test_split
    X, y = dataset, labels
    train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=123)
    return train_dataset, test_dataset, train_labels, test_labels

In [ ]:
def visualizeDataset(dataset, column_names, percent):
    visualize_dataset = raw_dataset.sample(frac=percent)
    sns.pairplot(visualize_dataset[column_names], diag_kind="kde")

In [ ]:
def scaleVolume(dataset, factor):
    dataset['volume'] = dataset['volume'] / factor
    return dataset

In [4]:
def normDataset(x,stats):
  return (x - stats['mean']) / stats['std']

In [11]:
def evaluatePerf(y, preds):
    def flatten(l): 
        if (isinstance(l[0], list)): return [item for sublist in l for item in sublist]
        else: return l
    import math
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    y = flatten(y)
    preds = flatten(preds)
    y = [a for a,b in zip(y,preds) if not math.isnan(b)]
    preds = [a for a in preds if not math.isnan(a)]
    rmse = mean_squared_error(y, preds)
    rmae = mean_absolute_error(y, preds)
    r2 = r2_score(y, preds)
    print("MSE, MAE, r2: %f,%f,%f" % (rmse, rmae, r2))

In [5]:
def worstOffenders(y, preds, data, labels):
    import numpy as np
    mae = np.abs(y - preds)
    mse = (y - preds) * (y - preds)
    mae_mean, mae_std = np.mean(mae), np.std(mae)
    mse_mean, mse_std = np.mean(mse), np.std(mse)
    print('mae mean: %.3f, stddev: %.3f' % (mae_mean, mae_std))
    print('mse mean: %.3f, stddev: %.3f' % (mse_mean, mse_std))
    print('MAE Outliers:')
    for i,x in enumerate(mae):
        if ((x - mae_mean) > 3 * mae_std):
            print('Error: %.1f (want %.1f, got %.1f), file %d, row %d, _idx %d %d, _val %d %d, volume %f' % (
                                                                                            y.iloc[i] - preds[i],
                                                                                            y.iloc[i], preds[i],
                                                                                            data.iloc[i]['fileId'],
                                                                                            data.iloc[i]['row'],
                                                                                            data.iloc[i]['rising_idx'],
                                                                                            data.iloc[i]['falling_idx'],
                                                                                            data.iloc[i]['first_val'],
                                                                                            data.iloc[i]['last_val'],
                                                                                            data.iloc[i]['volume']))

In [15]:
def evaluateCustom(model, df2, truth):
    print(model.predict(df2))
    print('True label = %f' % truth)

In [16]:
def extractNNWeights(model, modelName, hDims, column_names, train_stats):
    i = 0

    print('  def %s_mean[I,T](toI: scala.Double => I, toT: scala.Double => T) = (toI(%d),toI(%f),toI(%f),toI(%f),toT(%f),toT(%f),toT(%f),toT(%f))' % (modelName, train_stats['mean']['row'],
                                                                                                                                                            train_stats['mean']['rising_idx'],
                                                                                                                                                            train_stats['mean']['falling_idx'],
                                                                                                                                                            train_stats['mean']['volume'],
                                                                                                                                                            train_stats['mean']['rising_weight'],
                                                                                                                                                            train_stats['mean']['falling_weight'],
                                                                                                                                                            train_stats['mean']['first_val'],
                                                                                                                                                            train_stats['mean']['last_val']))
    #activation(dot(input, kernel) + bias)
    assembled = []
    for layer in model.get_weights():
        s = 'W' if (i % 2 == 0) else 'B'
        vals = []
        for j,x in enumerate(layer.flatten()):
            if (i // 2 >= len(hDims)): dim = 1
            else: dim = hDims[i//2]
            if (i % 2 == 0):
                if (i == 0):
                    ft = column_names[:-1][int(j / dim)]
                    scaled = x / train_stats['std'][ft]
                else:
                    scaled = x
                vals.append('%f' % scaled)
            else:
                vals.append('%f' % x)
        print('  private val %s_l%d%s = Seq(%s)' % (modelName, 1+i//2, s, ','.join(vals)))
    #     print('Layer %d' % i)
    #     print(layer)
        assembled.append('%s_l%d%s.map(cast)' % (modelName, 1+i//2, s))
        i =  i + 1


    print('  def %s_layers[T](cast: scala.Double => T) = (%s)' % (modelName, ','.join(assembled)))

In [ ]:
def extractXGWeights(xg_reg, modelName, MAX_DEPTH):
    import pickle
    import json
    xg_reg.get_booster().dump_model('example_params')
#     xg_reg.dump_model('example_params')
    # xg_reg.get_booster().get_dump()
    numlines = len(open('example_params').readlines(  ))

    import numpy
    import math
    fields = {}
    threshes = {}
    lefts = {}
    rights = {}
    tree = -1
    num_nodes = int(math.pow(2,MAX_DEPTH+1))

    with open('example_params') as file:
        for cnt, line in enumerate(file):
            if (not 'booster' in line):
                idx = int(line.strip().split(':')[0])
                if ('leaf=' in line):
                    thresh = float(line.strip().split('leaf=')[-1])
                    fields[idx] = ''
                    threshes[idx] = thresh
                else:
                    fields[idx] = line.strip().split('[')[1].split('<')[0]
                    threshes[idx] = float(line.strip().split('[')[1].split('<')[1].split(']')[0])
                    lefts[idx] = int(line.strip().split('yes=')[1].split(',')[0])
                    rights[idx] = int(line.strip().split('no=')[1].split(',')[0])
            if (('booster' in line and cnt != 0) or cnt == numlines-1):
                tree = tree + 1
                scala = ['("", 0)' for i in range(num_nodes)]

                # Decompress the tree into a complete tree
                leftsArr = [lefts[0]]
                rightsArr = [rights[0]]
                scala[0] = '("%s", %f)' % (fields[0], threshes[0])
                for idx in range(1,num_nodes):
                    if (idx % 2 == 1 and leftsArr[int(idx/2)] != None):
                        lookup = leftsArr[int(idx/2)]
                    elif (idx % 2 == 0 and rightsArr[int(idx/2)-1] != None):
                        lookup = rightsArr[int(idx/2)-1]
                    else: lookup = None

                    if (lookup != None):
                        scala[idx] = '("%s", %f)' % (fields[lookup], threshes[lookup])
                        if (lookup in lefts):
                            leftsArr.append(lefts[lookup])
                        else:
                            leftsArr.append(None)
                        if (lookup in rights):
                            rightsArr.append(rights[lookup])
                        else:
                            rightsArr.append(None)
                    else:
                        leftsArr.append(None)
                        rightsArr.append(None)


                print('  private val %s_tree%d: List[(String,Double)] = List(' % (modelName,tree) + ','.join(scala).replace(' ','') + ')')
                fields = {}
                threshes = {}
                lefts = {}
                rights = {}

    print('  def %s_trees[T](cast: scala.Double => T) = List(%s)' % (modelName,','.join(['%s_tree%d.map{x => (x._1, cast(x._2))}' % (modelName,d) for d in range(tree+1)])))
#     print('  def base_score = %f' % BASE_SCORE)
    #         print("Line {}: {}".format(cnt, line))



In [ ]:
def extractLatticeWeights(model, modelName):
    print('TBD :)')


In [ ]:
def randomImageIndices(numImages, percent):
    import random
    random.seed(1337)
    opts = list(range(0,numImages))
    selection = []
    for i in range(0,int(numImages*percent)):
        r=random.randint(0,len(opts)-1)
        selection.append(opts.pop(r))
    return selection
        
        

In [ ]:
def rowOffsets():
#     datadir = '/scratch/mattfel/data-fs/raw_timetool/u1/coffee/2dtimetool_simulation_data/raw/'

#     raw_labels = []
#     for fileId in range(1,20): #range(1,99999)::
#         fileType = 'fibermap'
#         if (fileId % 100 == 0): print('at %d' % fileId)
#         f = datadir + '/chirp-2000_interferedelay1650_photonen6.0_carriertagdiamond_nfibers109_netalon1_1.00_1.00_%s.out.%d' % (fileType,fileId)
#         if (path.exists(f)):
#             rawdata = pd.read_csv(f, 
#                               skiprows = 1, usecols = [6], 
#                               na_values = "?", comment='\t',
#                               sep="\t|,", skipinitialspace=True)
#             raw_labels.append(rawdata.as_matrix(columns=['delay']).flatten())
#     #         raw_labels = raw_labels.append(rawdata)
#         else:
#             print('file %s does not exist!' % f)

#     deltas = []
#     for r in raw_labels:
#         arr = r[0] - r
#         deltas.append(arr)
#     deltas = np.asmatrix(deltas)
#     print(deltas[0])
#     return     [ 0.0, -366.63, -183.32, 183.31, 366.63, 183.31, -183.32, -733.26, -366.63, 366.63, 733.26, 366.63, -366.63, -549.95, 0.0, 549.94, 549.94, 0.0, -549.95, -1099.895, -549.95, 549.94, 1099.89, 549.94, -549.95, -916.58, -183.32, 733.26, 916.57, 183.31, -733.26, -916.58, -733.26, 183.31, 916.57, 733.26, -183.32, -1466.525, -733.26, 733.26, 1466.52, 733.26, -733.26, -1283.21, -366.63, 916.57, 1283.2, 366.63, -916.58, -1283.21, -916.58, 366.63, 1283.2, 916.57, -366.63, -1099.895, 0.0, 1099.89, 1099.89, 0.0, -1099.895, -1833.15477, -916.58, 916.57, 1833.15, 916.57, -916.58, -1649.84, -549.95, 1099.89, 1649.83, 549.94, -1099.895, -1649.84, -1099.895, 549.94, 1649.83, 1099.89, -549.95, -1283.21, 183.31, 1466.52, 1283.2, -183.32, -1466.525, -1283.21, -1466.525, -183.32, 1283.2, 1466.52, 183.31, -1649.84, 0.0, 1649.83, 1649.83, 0.0, -1649.84, -1833.15479, -366.63, 1466.52, 1833.15, 366.63, -1466.525, -1833.15475, 1466.525, 366.63, 1833.15, 1466.52, -366.63, ]
    return [0.000,-366.630,-183.561,183.066,366.626,183.556,-183.069,-733.260,-367.122,366.136,733.256,367.126,-366.138,-550.191,-0.492,549.696,550.186,0.492,-549.699,-1099.890,-550.682,549.206,1099.886,550.686,-549.207,-916.821,-184.052,732.766,916.816,184.056,-732.768,-916.329,-733.751,182.576,916.326,733.756,-182.578,-1466.520,-734.243,732.276,1466.516,734.246,-732.277,-1283.451,-367.613,915.836,1283.446,367.616,-915.837,-1282.959,-917.312,365.646,1282.956,917.316]

In [ ]:
def predictPoly(pt, rising_model, falling_model):
    import numpy as np
    import math
    scale = 128.0 # divide rising/falling idx by this number so it fits in fpga precision

    tyr = np.sum([m * math.pow(pt['rising_idx'] / scale,i) for i,m in enumerate(rising_model[::-1])])
    tyf = np.sum([m * math.pow(pt['falling_idx'] / scale,i) for i,m in enumerate(falling_model[::-1])])
    
    if (pt['first_val'] >= 30): return tyf # only use falling
    elif (pt['last_val'] >= 30): return tyr # only use rising
    return (tyr + tyf) / 2 # use both rising and falling